In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances

In [30]:
df = pd.read_csv('data/megalist.csv')

In [31]:
df.head()

,title,0,1,reviews
0,Counter-Strike: Global Offensive,Action,730,['Your team in every random competitive game:\...
1,Grand Theft Auto V,Action,271590,"['Trevor Philips Enterprises 🥵', 'After Ep*c G..."
2,Tom Clancy's Rainbow Six® Siege,Action,359550,['> Play as French guy with giant shield\n> St...
3,Left 4 Dead 2,Action,550,"['h', 'why did they leave me for dead for that..."
4,Warframe,Action,230410,"['look at my hours', 'look at my hours']"


In [32]:
megalist=[]

for rows in df.reviews:
    for items in rows:
        megalist.append(items)

In [33]:
len(megalist)

1720190

In [16]:
df

,title,0,1,reviews
0,Counter-Strike: Global Offensive,Action,730,['Your team in every random competitive game:\...
1,Grand Theft Auto V,Action,271590,"['Trevor Philips Enterprises 🥵', 'After Ep*c G..."
2,Tom Clancy's Rainbow Six® Siege,Action,359550,['> Play as French guy with giant shield\n> St...
3,Left 4 Dead 2,Action,550,"['h', 'why did they leave me for dead for that..."
4,Warframe,Action,230410,"['look at my hours', 'look at my hours']"
...,...,...,...,...
120,Factorio,Strategy,427520,['Dev pisses off Reddit. Worth every cent for ...
121,Total War: WARHAMMER II,Strategy,594570,"[""In Sigmar's name YES!"", 'A little bit of a l..."
122,Hearts of Iron IV,Strategy,394360,"['virginity is awesome 😎😎😎😎😎', 'nobody will ev..."
123,STAR WARS™ Empire at War - Gold Pack,Strategy,32470,"[""Amazing game. The graphics are potato but th..."


In [25]:
df.iloc[0]['reviews'][1]

"'"